In [14]:
# Install what we need
!pip install --quiet --upgrade \
    datasets \
    sentence-transformers \
    faiss-cpu \
    "openai>=1.0.0" #openai \
    rouge_score \
    gradio

In [15]:
!pip install "openai==0.28"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [1]:
# A hand‑picked set of 6 abstracts, each with an ID, title, and abstract text
docs = [
    {"id":"D1","title":"Photosynthesis Basics",
     "abstract":"Photosynthesis converts light energy into chemical energy in plants."},
    {"id":"D2","title":"Quantum Entanglement",
     "abstract":"Entangled particles share state information instantly across distances."},
    {"id":"D3","title":"Federated Learning Overview",
     "abstract":"Federated learning trains models across decentralized devices without sharing raw data."},
    {"id":"D4","title":"CRISPR Gene Editing",
     "abstract":"CRISPR enables precise editing of DNA sequences using Cas9 enzymes."},
    {"id":"D5","title":"Black Hole Formation",
     "abstract":"Black holes form when massive stars collapse under their own gravity."},
    {"id":"D6","title":"Neural Network Fundamentals",
     "abstract":"Neural networks are layers of interconnected nodes inspired by the human brain."},
]


In [13]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Embed abstracts
embedder = SentenceTransformer("allenai-specter")
texts = [d["abstract"] for d in docs]
embs = embedder.encode(texts, convert_to_numpy=True)
faiss.normalize_L2(embs)

# 2. Build FAISS index (inner product on normalized vectors = cosine similarity)
dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embs)

print(f"Indexed {index.ntotal} docs ✔️")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/462k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexed 6 docs ✔️


In [14]:
def retrieve(query, k=3):
    # 1) Embed & normalize the query
    qv = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(qv)
    # 2) Search top‑k
    scores, ids = index.search(qv, k)
    return [
        {
          "id":     docs[int(i)]["id"],
          "title":  docs[int(i)]["title"],
          "abstract": docs[int(i)]["abstract"],
          "score":  float(scores[0, idx])
        }
        for idx, i in enumerate(ids[0])
    ]

# Quick check
for r in retrieve("Explain federated learning", k=2):
    print(f"[{r['score']:.3f}] {r['id']} — {r['title']}")


[0.867] D3 — Federated Learning Overview
[0.725] D6 — Neural Network Fundamentals


In [16]:
# Cell 4.1: Build BM25 and Cross‑Encoder reranker
!pip install rank_bm25
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder

# BM25 index
tokenized = [d["abstract"].lower().split() for d in docs]
bm25 = BM25Okapi(tokenized)

# Cross‑encoder reranker
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def retrieve_hybrid(query, k=2):
    sem = retrieve(query, k=10)
    sem_ids = [d["id"] for d in sem]
    bm25_scores = bm25.get_scores(query.lower().split())
    # attach bm25 score and rerank
    for d in sem:
        d["bm25_score"] = bm25_scores[docs.index(d)]
    sem.sort(key=lambda x: x["bm25_score"], reverse=True)
    return sem[:k]

def retrieve_rerank(query, k=2):
    sem = retrieve(query, k=10)
    pairs = [(query, d["abstract"]) for d in sem]
    ce_scores = reranker.predict(pairs)
    for d, s in zip(sem, ce_scores):
        d["rerank_score"] = s
    sem.sort(key=lambda x: x["rerank_score"], reverse=True)
    return sem[:k]


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

In [17]:
def retrieve_threshold(query, k=2, thresh=0.3):
    all_hits = retrieve(query, k=10)
    filtered = [h for h in all_hits if h["score"] >= thresh]
    return filtered[:k]

In [7]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [8]:
import os, openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
import openai, os
openai.api_key = os.getenv("OPENAI_API_KEY")  # ensure your env var is set

def generate_answer(query, retrieved, model="gpt-3.5-turbo"):
    # Build numbered context
    blocks = []
    for i, d in enumerate(retrieved, start=1):
        blocks.append(f"[{i}] Title: {d['title']}\nAbstract: {d['abstract']}")
    context = "\n\n".join(blocks)

    # Instruct to cite
    system = (
        "You are an academic assistant. Answer using ONLY the provided abstracts. "
        "After each fact, cite the source in brackets, e.g. [1]."
    )
    user_msg = f"Context:\n{context}\n\nQuestion: {query}"

    # Call the LLM
    resp = openai.ChatCompletion.create(
      model=model,
      messages=[
        {"role":"system","content":system},
        {"role":"user","content":user_msg}
      ],
      temperature=0
    )
    return resp.choices[0].message.content

# Test
q = "What is federated learning?"
docs_ret = retrieve(q, k=3)
print(generate_answer(q, docs_ret))


Federated learning trains models across decentralized devices without sharing raw data. [1]


In [10]:
from rouge_score import rouge_scorer

# A small test set with gold‐standard answers & relevant doc IDs
tests = [
  {"query":"What is federated learning?",
   "relevant":["D3"],
   "ref":"Federated learning trains models across decentralized devices without sharing raw data."},
  {"query":"How do black holes form?",
   "relevant":["D5"],
   "ref":"Black holes form when massive stars collapse under their own gravity."},
  {"query":"Explain photosynthesis.",
   "relevant":["D1"],
   "ref":"Photosynthesis converts light energy into chemical energy in plants."},
]

scorer = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

results = []
for t in tests:
    # Retrieval
    recs = retrieve(t["query"], k=2)
    rec_ids = [r["id"] for r in recs]
    prec_at_2 = len(set(rec_ids)&set(t["relevant"]))/2

    # Generation (use only top‐1)
    ans = generate_answer(t["query"], [recs[0]])
    rouge1 = scorer.score(t["ref"], ans)["rouge1"].fmeasure

    results.append({
      "query":t["query"],
      "Precision@2":round(prec_at_2,2),
      "Rouge‑1 F1": round(rouge1,2),
      "Answer": ans.replace("\n"," ")
    })

import pandas as pd
df = pd.DataFrame(results)
print(df.to_markdown(index=False))


| query                       |   Precision@2 |   Rouge‑1 F1 | Answer                                                                                               |
|:----------------------------|--------------:|-------------:|:-----------------------------------------------------------------------------------------------------|
| What is federated learning? |           0.5 |         0.96 | Federated learning trains models across decentralized devices without sharing raw data. [1]          |
| How do black holes form?    |           0.5 |         0.96 | Black holes form when massive stars collapse under their own gravity. [1]                            |
| Explain photosynthesis.     |           0.5 |         0.72 | Photosynthesis is the process by which light energy is converted into chemical energy in plants. [1] |


In [21]:
# Cell 6: Evaluation with  Precision@2 & Rouge‑1 F1

from rouge_score import rouge_scorer
import pandas as pd

# 1) Define precision metrics
def precision_at_1(retrieved_ids, relevant_ids):
    return int(retrieved_ids[0] in relevant_ids)

def precision_at_2(retrieved_ids, relevant_ids):
    return len(set(retrieved_ids) & set(relevant_ids)) / 2

# 2) Prepare ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

# 3) Evaluate each test case
results = []
for t in tests:
    # (a) Retrieval: use your preferred method (e.g., retrieve_rerank or retrieve_hybrid)
    recs = retrieve_rerank(t["query"], k=2)
    rec_ids = [d["id"] for d in recs]

    # (b) Compute Precision@1 and Precision@2
    p1 = precision_at_1(rec_ids, t["relevant"])
    p2 = precision_at_2(rec_ids, t["relevant"])

    # (c) Generate answer using top‑1 document
    ans = generate_answer(t["query"], recs[:1])

    # (d) Compute Rouge‑1 F1 against reference answer
    r1 = scorer.score(t["ref"], ans)["rouge1"].fmeasure

    results.append({
        "query":       t["query"],
        "P@2":         round(p2, 2),
        "Rouge‑1 F1":  round(r1, 2),
        "answer":      ans.replace("\n", " ")
    })

# 4) Display as a DataFrame
df = pd.DataFrame(results)
df


,query,P@2,Rouge‑1 F1,answer
0,What is federated learning?,0.5,0.81,Federated learning is a method that trains mod...
1,How do black holes form?,0.5,0.96,Black holes form when massive stars collapse u...
2,Explain photosynthesis.,0.5,0.40,Photosynthesis is the process by which plants ...


In [28]:
# Cell 8 (revised): Gradio UI with working day/night toggle (no sanitize parameter)

import gradio as gr

# 1) CSS for themes & layout
demo_css = """
#app {
    --bg-color: #f9f9f9;
    --text-color: #333;
    --chat-bg: #fff;
    --source-bg: #f1f1f1;
    --button-bg: #e0e0e0;
    --button-hover: #ccc;
    background-color: var(--bg-color);
    color: var(--text-color);
    padding: 1rem;
    font-family: Arial, sans-serif;
    position: relative;
}
#app.dark {
    --bg-color: #2b2b2b;
    --text-color: #ddd;
    --chat-bg: #3b3b3b;
    --source-bg: #444;
    --button-bg: #555;
    --button-hover: #666;
}
#app #chatbot .overflow-y-auto {
    height: 400px;
    background-color: var(--chat-bg);
    color: var(--text-color);
    border-radius: 0.5rem;
    padding: 0.5rem;
}
#app .source-list {
    background-color: var(--source-bg);
    padding: 1rem;
    border-radius: 0.5rem;
    list-style-position: inside;
}
#app .source-list li a {
    color: var(--text-color);
    text-decoration: none;
}
#theme-toggle {
    position: absolute;
    top: 1rem;
    right: 1rem;
    background-color: var(--button-bg);
    color: var(--text-color);
    border: none;
    padding: 0.5rem 0.75rem;
    border-radius: 0.5rem;
    cursor: pointer;
    font-size: 1.25rem;
}
#theme-toggle:hover {
    background-color: var(--button-hover);
}
"""

# 2) HTML + JS for the toggle button
toggle_html = """
<button id="theme-toggle" title="Switch to dark mode to reduce eye strain"
        aria-label="Toggle light/dark mode">🌙</button>
<script>
  const btn = document.getElementById('theme-toggle');
  btn.addEventListener('click', () => {
    const app = document.getElementById('app');
    if (app.classList.toggle('dark')) {
      btn.textContent = '☀️';
      btn.title = 'Switch to light mode to reduce eye strain';
    } else {
      btn.textContent = '🌙';
      btn.title = 'Switch to dark mode to reduce eye strain';
    }
  });
</script>
"""

def qa_interactive(query, history, k):
    history = history + [{"role": "user", "content": query}]
    retrieved = retrieve_rerank(query, k=k)
    answer = generate_answer(query, retrieved)
    history = history + [{"role": "assistant", "content": answer}]
    sources_html = "<ul class='source-list'>"
    for i, doc in enumerate(retrieved, start=1):
        href = f"https://arxiv.org/abs/{doc['id']}"
        sources_html += (
            f"<li><a href='{href}' target='_blank'>{i}. {doc['title']}</a>"
            f" (score: {doc['score']:.2f})</li>"
        )
    sources_html += "</ul>"
    return history, "", sources_html

with gr.Blocks(css=demo_css, elem_id="app") as demo:
    # Inject toggle HTML (script will run without sanitize)
    gr.HTML(toggle_html)
    gr.Markdown("## Interactive RAG System")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="Conversation", type="messages")
            user_input = gr.Textbox(
                label="Your Question",
                placeholder="Type your research question here…",
                lines=1
            )
            k_slider = gr.Slider(
                minimum=1, maximum=5, value=3, step=1,
                label="Number of sources to retrieve (k)"
            )
            submit_btn = gr.Button("🔍 Ask")

        with gr.Column(scale=1):
            gr.Markdown("### Retrieved Sources")
            source_output = gr.HTML()

    submit_btn.click(
        fn=qa_interactive,
        inputs=[user_input, chatbot, k_slider],
        outputs=[chatbot, user_input, source_output]
    )
    user_input.submit(
        fn=qa_interactive,
        inputs=[user_input, chatbot, k_slider],
        outputs=[chatbot, user_input, source_output]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e2fbac38625746abe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import gradio as gr

def qa_ui(query):
    docs_ret = retrieve(query, k=3)
    ans = generate_answer(query, docs_ret)
    refs = "\n".join([f"[{i+1}] {d['title']}" for i,d in enumerate(docs_ret)])
    return ans, refs

iface = gr.Interface(
    fn=qa_ui,
    inputs=gr.Textbox(lines=2, placeholder="Ask a research question…"),
    outputs=[gr.Markdown(), gr.Markdown()],
    title="Corpus RAG Demo",
    description="Answers with citation-backed abstracts from a 6‑doc corpus."
)
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b29884a9f02d724ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
